# Lab Exercise: SQL Analysis with Polars

In this lab, you'll practice SQL queries using Polars' built-in SQL functionality. Complete each exercise by writing the appropriate SQL query.

In [3]:
# Setup - Run this cell first
import polars as pl

READ_KW = dict(null_values=["NA", ""], infer_schema_length=10000)

airlines = pl.read_csv("https://raw.githubusercontent.com/philhetzel/opan5510-class11/refs/heads/main/data/nyc_airlines.csv", **READ_KW)
airports = pl.read_csv("https://raw.githubusercontent.com/philhetzel/opan5510-class11/refs/heads/main/data/nyc_airports.csv", **READ_KW)
flights  = pl.read_csv("https://raw.githubusercontent.com/philhetzel/opan5510-class11/refs/heads/main/data/nyc_flights.csv", **READ_KW)
planes   = pl.read_csv("https://raw.githubusercontent.com/philhetzel/opan5510-class11/refs/heads/main/data/nyc_planes.csv", **READ_KW)
weather  = pl.read_csv("https://raw.githubusercontent.com/philhetzel/opan5510-class11/refs/heads/main/data/nyc_weather.csv", **READ_KW)

# Parse time_hour como Datetime (tolerante)
flights = flights.with_columns(pl.col("time_hour").str.strptime(pl.Datetime, strict=False))
weather = weather.with_columns(pl.col("time_hour").str.strptime(pl.Datetime, strict=False))

# Criar SQL context
ctx = pl.SQLContext(
    airlines=airlines,
    airports=airports,
    flights=flights,
    planes=planes,
    weather=weather,
    eager_execution=True
)

print("Setup complete! Tables available:")
print(ctx.execute("SHOW TABLES"))

Setup complete! Tables available:
shape: (5, 1)
┌──────────┐
│ name     │
│ ---      │
│ str      │
╞══════════╡
│ airlines │
│ airports │
│ flights  │
│ planes   │
│ weather  │
└──────────┘


/tmp/ipython-input-200453716.py:17: DeprecationWarning: The argument `eager_execution` for `SQLContext.__init__` is deprecated. It has been renamed to `eager`.
  ctx = pl.SQLContext(


## Exercise 1: Basic Queries

### 1.1 Find all unique carriers in the airlines table

In [4]:
# Write your SQL query here
result = ctx.execute("""
SELECT DISTINCT carrier
FROM airlines
ORDER BY carrier
""")

print(result)

shape: (16, 1)
┌─────────┐
│ carrier │
│ ---     │
│ str     │
╞═════════╡
│ 9E      │
│ AA      │
│ AS      │
│ B6      │
│ DL      │
│ …       │
│ UA      │
│ US      │
│ VX      │
│ WN      │
│ YV      │
└─────────┘


### 1.2 Find the top 10 destinations by number of flights

In [5]:
# Write your SQL query here
result = ctx.execute("""
SELECT dest, COUNT(*) AS flights
FROM flights
GROUP BY dest
ORDER BY flights DESC
LIMIT 10
""")

print(result)

shape: (10, 2)
┌──────┬─────────┐
│ dest ┆ flights │
│ ---  ┆ ---     │
│ str  ┆ u32     │
╞══════╪═════════╡
│ ORD  ┆ 17283   │
│ ATL  ┆ 17215   │
│ LAX  ┆ 16174   │
│ BOS  ┆ 15508   │
│ MCO  ┆ 14082   │
│ CLT  ┆ 14064   │
│ SFO  ┆ 13331   │
│ FLL  ┆ 12055   │
│ MIA  ┆ 11728   │
│ DCA  ┆ 9705    │
└──────┴─────────┘


### 1.3 Find all flights that departed more than 2 hours late (120 minutes)

In [6]:
# Write your SQL query here
result = ctx.execute("""
SELECT *
FROM flights
WHERE dep_delay > 120
ORDER BY dep_delay DESC, year, month, day
""")

print(result)

shape: (9_723, 19)
┌──────┬───────┬─────┬──────────┬───┬──────────┬──────┬────────┬─────────────────────────┐
│ year ┆ month ┆ day ┆ dep_time ┆ … ┆ distance ┆ hour ┆ minute ┆ time_hour               │
│ ---  ┆ ---   ┆ --- ┆ ---      ┆   ┆ ---      ┆ ---  ┆ ---    ┆ ---                     │
│ i64  ┆ i64   ┆ i64 ┆ i64      ┆   ┆ i64      ┆ i64  ┆ i64    ┆ datetime[μs, UTC]       │
╞══════╪═══════╪═════╪══════════╪═══╪══════════╪══════╪════════╪═════════════════════════╡
│ 2013 ┆ 1     ┆ 9   ┆ 641      ┆ … ┆ 4983     ┆ 9    ┆ 0      ┆ 2013-01-09 14:00:00 UTC │
│ 2013 ┆ 6     ┆ 15  ┆ 1432     ┆ … ┆ 483      ┆ 19   ┆ 35     ┆ 2013-06-15 23:00:00 UTC │
│ 2013 ┆ 1     ┆ 10  ┆ 1121     ┆ … ┆ 719      ┆ 16   ┆ 35     ┆ 2013-01-10 21:00:00 UTC │
│ 2013 ┆ 9     ┆ 20  ┆ 1139     ┆ … ┆ 2586     ┆ 18   ┆ 45     ┆ 2013-09-20 22:00:00 UTC │
│ 2013 ┆ 7     ┆ 22  ┆ 845      ┆ … ┆ 589      ┆ 16   ┆ 0      ┆ 2013-07-22 20:00:00 UTC │
│ …    ┆ …     ┆ …   ┆ …        ┆ … ┆ …        ┆ …    ┆ …      ┆ …     

## Exercise 2: Aggregation

### 2.1 Calculate the average departure delay for each origin airport

In [7]:
# Write your SQL query here
result = ctx.execute("""
SELECT origin,
       ROUND(AVG(dep_delay), 2) AS avg_dep_delay
FROM flights
GROUP BY origin
ORDER BY avg_dep_delay DESC
""")

print(result)

shape: (3, 2)
┌────────┬───────────────┐
│ origin ┆ avg_dep_delay │
│ ---    ┆ ---           │
│ str    ┆ f64           │
╞════════╪═══════════════╡
│ EWR    ┆ 15.11         │
│ JFK    ┆ 12.11         │
│ LGA    ┆ 10.35         │
└────────┴───────────────┘


### 2.2 Find the busiest month of the year

Count the number of flights per month and find which month has the most flights.

In [8]:
result = ctx.execute("""
SELECT month, COUNT(*) AS flights
FROM flights
GROUP BY month
ORDER BY flights DESC
LIMIT 1
""")

print(result)

shape: (1, 2)
┌───────┬─────────┐
│ month ┆ flights │
│ ---   ┆ ---     │
│ i64   ┆ u32     │
╞═══════╪═════════╡
│ 7     ┆ 29425   │
└───────┴─────────┘


### 2.3 Calculate the on-time performance rate for each carrier

Consider a flight on-time if the departure delay is <= 15 minutes.

In [11]:
result = ctx.execute("""
SELECT
  airline,
  carrier,
  SUM(has_dep_delay) AS flights_measured,
  SUM(is_ontime)     AS on_time,
  ROUND(
    CASE WHEN SUM(has_dep_delay) = 0
         THEN NULL
         ELSE 100.0 * SUM(is_ontime) / SUM(has_dep_delay)
    END, 2
  ) AS on_time_rate_pct
FROM (
  SELECT
    a.name AS airline,
    f.carrier,
    CASE WHEN f.dep_delay <= 15 THEN 1 ELSE 0 END AS is_ontime,
    CASE WHEN f.dep_delay IS NULL THEN 0 ELSE 1 END AS has_dep_delay
  FROM flights AS f
  JOIN airlines AS a USING (carrier)
) AS s
GROUP BY airline, carrier
ORDER BY on_time_rate_pct DESC
""")

print(result)

shape: (16, 5)
┌─────────────────────────────┬─────────┬──────────────────┬─────────┬──────────────────┐
│ airline                     ┆ carrier ┆ flights_measured ┆ on_time ┆ on_time_rate_pct │
│ ---                         ┆ ---     ┆ ---              ┆ ---     ┆ ---              │
│ str                         ┆ str     ┆ i32              ┆ i32     ┆ f64              │
╞═════════════════════════════╪═════════╪══════════════════╪═════════╪══════════════════╡
│ Hawaiian Airlines Inc.      ┆ HA      ┆ 342              ┆ 318     ┆ 92.98            │
│ US Airways Inc.             ┆ US      ┆ 19873            ┆ 17453   ┆ 87.82            │
│ Alaska Airlines Inc.        ┆ AS      ┆ 712              ┆ 618     ┆ 86.8             │
│ American Airlines Inc.      ┆ AA      ┆ 32093            ┆ 26981   ┆ 84.07            │
│ Delta Air Lines Inc.        ┆ DL      ┆ 47761            ┆ 39967   ┆ 83.68            │
│ …                           ┆ …       ┆ …                ┆ …       ┆ …             

## Exercise 3: Joins

### 3.1 List all flights with their airline names (not just carrier codes)

Show the first 20 flights with carrier code, airline name, flight number, origin, and destination.

In [12]:
# Write your SQL query here
result = ctx.execute("""
SELECT
  f.carrier,
  a.name AS airline_name,
  f.flight,
  f.origin,
  f.dest
FROM flights AS f
JOIN airlines AS a USING (carrier)
ORDER BY f.year, f.month, f.day, f.dep_time, f.flight
LIMIT 20
""")

print(result)

shape: (20, 5)
┌─────────┬────────────────────────┬────────┬────────┬──────┐
│ carrier ┆ airline_name           ┆ flight ┆ origin ┆ dest │
│ ---     ┆ ---                    ┆ ---    ┆ ---    ┆ ---  │
│ str     ┆ str                    ┆ i64    ┆ str    ┆ str  │
╞═════════╪════════════════════════╪════════╪════════╪══════╡
│ UA      ┆ United Air Lines Inc.  ┆ 1545   ┆ EWR    ┆ IAH  │
│ UA      ┆ United Air Lines Inc.  ┆ 1714   ┆ LGA    ┆ IAH  │
│ AA      ┆ American Airlines Inc. ┆ 1141   ┆ JFK    ┆ MIA  │
│ B6      ┆ JetBlue Airways        ┆ 725    ┆ JFK    ┆ BQN  │
│ DL      ┆ Delta Air Lines Inc.   ┆ 461    ┆ LGA    ┆ ATL  │
│ …       ┆ …                      ┆ …      ┆ …      ┆ …    │
│ UA      ┆ United Air Lines Inc.  ┆ 1187   ┆ EWR    ┆ LAS  │
│ B6      ┆ JetBlue Airways        ┆ 1806   ┆ JFK    ┆ BOS  │
│ B6      ┆ JetBlue Airways        ┆ 371    ┆ LGA    ┆ FLL  │
│ MQ      ┆ Envoy Air              ┆ 4650   ┆ LGA    ┆ ATL  │
│ B6      ┆ JetBlue Airways        ┆ 343    ┆ EWR    ┆ 

### 3.2 Find the average age of planes for each carrier

Hint: The planes table has a `year` column for manufacture year. Calculate age based on 2013.

In [13]:
# Write your SQL query here
result = ctx.execute("""
WITH plane_age_per_carrier AS (
  SELECT DISTINCT
    a.name      AS airline,
    f.carrier   AS carrier,
    f.tailnum   AS tailnum,
    (2013 - p.year) AS age
  FROM flights AS f
  JOIN planes  AS p USING (tailnum)
  JOIN airlines AS a USING (carrier)
  WHERE f.tailnum IS NOT NULL
    AND p.year   IS NOT NULL
)
SELECT
  airline,
  carrier,
  ROUND(AVG(age), 2) AS avg_plane_age
FROM plane_age_per_carrier
GROUP BY airline, carrier
ORDER BY avg_plane_age DESC
""")

print(result)

shape: (16, 3)
┌────────────────────────┬─────────┬───────────────┐
│ airline                ┆ carrier ┆ avg_plane_age │
│ ---                    ┆ ---     ┆ ---           │
│ str                    ┆ str     ┆ f64           │
╞════════════════════════╪═════════╪═══════════════╡
│ Envoy Air              ┆ MQ      ┆ 35.5          │
│ American Airlines Inc. ┆ AA      ┆ 25.4          │
│ Delta Air Lines Inc.   ┆ DL      ┆ 17.67         │
│ United Air Lines Inc.  ┆ UA      ┆ 13.05         │
│ Southwest Airlines Co. ┆ WN      ┆ 11.01         │
│ …                      ┆ …       ┆ …             │
│ JetBlue Airways        ┆ B6      ┆ 6.5           │
│ Alaska Airlines Inc.   ┆ AS      ┆ 5.16          │
│ Frontier Airlines Inc. ┆ F9      ┆ 5.0           │
│ Virgin America         ┆ VX      ┆ 4.29          │
│ Hawaiian Airlines Inc. ┆ HA      ┆ 1.23          │
└────────────────────────┴─────────┴───────────────┘


### 3.3 Find flights that experienced both departure delays and bad weather

Join flights with weather data and find flights where departure delay > 30 minutes and either wind_speed > 20 or precip > 0.1

In [14]:
# First, explore the weather table structure
result = ctx.execute("""
SELECT
  f.year, f.month, f.day, f.dep_time,
  f.origin, f.dest,
  f.dep_delay,
  w.wind_speed, w.precip
FROM flights AS f
JOIN weather AS w
  ON f.origin = w.origin
 AND f.time_hour = w.time_hour
WHERE f.dep_delay > 30
  AND (w.wind_speed > 20 OR w.precip > 0.1)
ORDER BY f.dep_delay DESC, f.year, f.month, f.day
""")

print(result)

shape: (4_938, 9)
┌──────┬───────┬─────┬──────────┬───┬──────┬───────────┬────────────┬────────┐
│ year ┆ month ┆ day ┆ dep_time ┆ … ┆ dest ┆ dep_delay ┆ wind_speed ┆ precip │
│ ---  ┆ ---   ┆ --- ┆ ---      ┆   ┆ ---  ┆ ---       ┆ ---        ┆ ---    │
│ i64  ┆ i64   ┆ i64 ┆ i64      ┆   ┆ str  ┆ i64       ┆ f64        ┆ f64    │
╞══════╪═══════╪═════╪══════════╪═══╪══════╪═══════════╪════════════╪════════╡
│ 2013 ┆ 4     ┆ 10  ┆ 1100     ┆ … ┆ TPA  ┆ 960       ┆ 31.07106   ┆ 0.11   │
│ 2013 ┆ 12    ┆ 14  ┆ 830      ┆ … ┆ TPA  ┆ 825       ┆ 20.71404   ┆ 0.01   │
│ 2013 ┆ 4     ┆ 19  ┆ 617      ┆ … ┆ LAS  ┆ 797       ┆ 25.31716   ┆ 0.0    │
│ 2013 ┆ 4     ┆ 19  ┆ 606      ┆ … ┆ IAH  ┆ 761       ┆ 25.31716   ┆ 0.0    │
│ 2013 ┆ 4     ┆ 10  ┆ 604      ┆ … ┆ MCO  ┆ 639       ┆ 33.37262   ┆ 0.14   │
│ …    ┆ …     ┆ …   ┆ …        ┆ … ┆ …    ┆ …         ┆ …          ┆ …      │
│ 2013 ┆ 12    ┆ 6   ┆ 2036     ┆ … ┆ DAY  ┆ 31        ┆ 14.96014   ┆ 0.14   │
│ 2013 ┆ 12    ┆ 22  ┆ 731      ┆ 

## Exercise 4: Advanced Queries

### 4.1 Find the most popular aircraft types (by number of flights)

Join flights with planes to get manufacturer and model information. Show top 10.

In [15]:
# Write your SQL query here
result = ctx.execute("""
SELECT
  p.manufacturer,
  p.model,
  COUNT(*) AS flights
FROM flights AS f
JOIN planes  AS p USING (tailnum)
GROUP BY p.manufacturer, p.model
ORDER BY flights DESC
LIMIT 10
""")

print(result)

shape: (10, 3)
┌───────────────────────────────┬─────────────────┬─────────┐
│ manufacturer                  ┆ model           ┆ flights │
│ ---                           ┆ ---             ┆ ---     │
│ str                           ┆ str             ┆ u32     │
╞═══════════════════════════════╪═════════════════╪═════════╡
│ AIRBUS                        ┆ A320-232        ┆ 31278   │
│ EMBRAER                       ┆ EMB-145LR       ┆ 28027   │
│ EMBRAER                       ┆ ERJ 190-100 IGW ┆ 23716   │
│ AIRBUS INDUSTRIE              ┆ A320-232        ┆ 14553   │
│ EMBRAER                       ┆ EMB-145XR       ┆ 14051   │
│ BOEING                        ┆ 737-824         ┆ 13809   │
│ BOMBARDIER INC                ┆ CL-600-2D24     ┆ 11807   │
│ BOEING                        ┆ 737-7H4         ┆ 10389   │
│ BOEING                        ┆ 757-222         ┆ 9150    │
│ MCDONNELL DOUGLAS AIRCRAFT CO ┆ MD-88           ┆ 8932    │
└───────────────────────────────┴─────────────────┴────

### 4.2 Analyze route performance

Find the top 10 routes (origin-destination pairs) with:
- Total number of flights
- Average departure delay
- Percentage of flights delayed more than 30 minutes

Include airport names, not just codes.

In [19]:
# Write your SQL query here
result = ctx.execute("""
WITH routes AS (
  SELECT
    f.origin,
    f.dest,
    f.dep_delay,
    ao.name AS origin_name,
    ad.name AS dest_name
  FROM flights AS f
  LEFT JOIN airports AS ao ON f.origin = ao.faa
  LEFT JOIN airports AS ad ON f.dest   = ad.faa
)
SELECT
  origin,
  origin_name,
  dest,
  dest_name,
  COUNT(*) AS total_flights,
  ROUND(AVG(dep_delay), 2) AS avg_dep_delay,
  ROUND(100.0 * AVG(CASE WHEN dep_delay > 30 THEN 1 ELSE 0 END), 2) AS pct_delayed_gt_30
FROM routes
GROUP BY origin, origin_name, dest, dest_name
ORDER BY total_flights DESC
LIMIT 10
""")

print(result)

shape: (10, 7)
┌────────┬────────────────┬──────┬────────────────┬───────────────┬───────────────┬────────────────┐
│ origin ┆ origin_name    ┆ dest ┆ dest_name      ┆ total_flights ┆ avg_dep_delay ┆ pct_delayed_gt │
│ ---    ┆ ---            ┆ ---  ┆ ---            ┆ ---           ┆ ---           ┆ _30            │
│ str    ┆ str            ┆ str  ┆ str            ┆ u32           ┆ f64           ┆ ---            │
│        ┆                ┆      ┆                ┆               ┆               ┆ f64            │
╞════════╪════════════════╪══════╪════════════════╪═══════════════╪═══════════════╪════════════════╡
│ JFK    ┆ John F Kennedy ┆ LAX  ┆ Los Angeles    ┆ 11262         ┆ 8.52          ┆ 9.83           │
│        ┆ Intl           ┆      ┆ Intl           ┆               ┆               ┆                │
│ LGA    ┆ La Guardia     ┆ ATL  ┆ Hartsfield     ┆ 10263         ┆ 11.45         ┆ 12.25          │
│        ┆                ┆      ┆ Jackson        ┆               ┆         

## Bonus: Compare with Polars

### Choose one of the queries above and implement it using Polars

This will help you understand the relationship between SQL and Polars operations.

In [20]:
# Example: Let's implement Exercise 2.1 (average delay by origin) in Polars

# SQL version (for reference)
sql_result = ctx.execute("""
    SELECT
        origin,
        AVG(dep_delay) as avg_delay
    FROM flights
    WHERE dep_delay IS NOT NULL
    GROUP BY origin
    ORDER BY avg_delay DESC
""")

# Polars version
polars_result = (
    flights
    .filter(pl.col('dep_delay').is_not_null())
    .group_by('origin')
    .agg(pl.col('dep_delay').mean().alias('avg_delay'))
    .sort('avg_delay', descending=True)
)

print("SQL Result:")
print(sql_result)
print("\nPolars Result:")
print(polars_result)

# Now implement one of your own queries in Polars below:
# Your Polars code here



# 1) Prepare airport name lookups for origin and dest
ao = airports.select(
    pl.col("faa").alias("origin"),
    pl.col("name").alias("origin_name")
)
ad = airports.select(
    pl.col("faa").alias("dest"),
    pl.col("name").alias("dest_name")
)

# 2) Join airport names onto flights
routes = (
    flights
    .join(ao, on="origin", how="left")
    .join(ad, on="dest",   how="left")
)

# 3) Aggregate metrics by route
polars_result_42 = (
    routes
    .group_by(["origin", "origin_name", "dest", "dest_name"])
    .agg([
        pl.len().alias("total_flights"),
        pl.col("dep_delay").mean().round(2).alias("avg_dep_delay"),
        (
            pl.when(pl.col("dep_delay") > 30).then(1).otherwise(0)
        ).mean().mul(100).round(2).alias("pct_delayed_gt_30")
    ])
    .sort("total_flights", descending=True)
    .head(10)
)

print(polars_result_42)

SQL Result:
shape: (3, 2)
┌────────┬───────────┐
│ origin ┆ avg_delay │
│ ---    ┆ ---       │
│ str    ┆ f64       │
╞════════╪═══════════╡
│ EWR    ┆ 15.107954 │
│ JFK    ┆ 12.112159 │
│ LGA    ┆ 10.346876 │
└────────┴───────────┘

Polars Result:
shape: (3, 2)
┌────────┬───────────┐
│ origin ┆ avg_delay │
│ ---    ┆ ---       │
│ str    ┆ f64       │
╞════════╪═══════════╡
│ EWR    ┆ 15.107954 │
│ JFK    ┆ 12.112159 │
│ LGA    ┆ 10.346876 │
└────────┴───────────┘
shape: (10, 7)
┌────────┬────────────────┬──────┬────────────────┬───────────────┬───────────────┬────────────────┐
│ origin ┆ origin_name    ┆ dest ┆ dest_name      ┆ total_flights ┆ avg_dep_delay ┆ pct_delayed_gt │
│ ---    ┆ ---            ┆ ---  ┆ ---            ┆ ---           ┆ ---           ┆ _30            │
│ str    ┆ str            ┆ str  ┆ str            ┆ u32           ┆ f64           ┆ ---            │
│        ┆                ┆      ┆                ┆               ┆               ┆ f64            │
╞════════╪